## Content
1. [Loading preprocessed data](#1.-Loading-preprocessed-data)  
2. [LR using pretrained word embeddings](#2.-LR-using-pretrained-word-embeddings)  
3. [BM25](#3.-BM25)
4. [String kernels](#4.-String-kernels)
5. [Classification {combined methods}](#5.-Classification-{combined-methods})
6. [Evaluation](#6.-Evaluation)

## 1. Loading preprocessed data

In [1]:
import evaluation
import json
import gensim
import numpy as np
import pandas as pd

from input_output import load_data, write_results
from pprint import pprint
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

IN_PATH = '../data/squad/'
WORD2VEC_PATH = '../word_embeddings/GoogleNews-vectors-negative300.bin'
word2vec_model = None

In [2]:
train = load_data(IN_PATH + 'train-v1.1-preprocessed.json')
dev = load_data(IN_PATH + 'dev-v1.1-preprocessed.json')

## 2. LR using pretrained word embeddings

In [3]:
def get_pretrained_word_embeddings(sentence, word_emb_dims=300):
    global word2vec_model
    
    if word2vec_model is None:
        print('load word2vec_model')
        word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(WORD2VEC_PATH, binary=True)

    word_embeddings = np.zeros(word_emb_dims)
    word_emb_nr = 0

    for word in sentence:
        if word2vec_model.vocab.has_key(word):
            word_embeddings += word2vec_model.word_vec(word)
            word_emb_nr += 1

    # a sentence is represented by the average of word_embbedings
    if word_emb_nr != 0:
        word_embeddings = word_embeddings / word_emb_nr

    return word_embeddings

In [4]:
def get_features(dataset, q_a_combination_method):
    train_x = list()
    train_y = list()
    train_steps = list()
    train_labels = list()
    
    for article in tqdm(dataset['data']):
        for qas_context in article['paragraphs']:
            for qas in qas_context['qas']:
                question_words = qas['question_words']
                question_embedding = get_pretrained_word_embeddings(question_words)
                
                candidate_sentences_embeddings = list()
                for sentence in qas_context['context_sentences_words']:
                    candidate_sentences_embeddings.append(get_pretrained_word_embeddings(sentence))
                
                labels = set([d['answer_label'] for d in qas['answers']])
                train_labels.append(labels)
                
                train_steps.append(len(candidate_sentences_embeddings))
                
                for i in range(len(candidate_sentences_embeddings)):
                    feature_vector = []
                    if q_a_combination_method == 'concatenation':
                        feature_vector = np.concatenate((question_embedding,
                                                         candidate_sentences_embeddings[i]))
                    elif q_a_combination_method == 'diff_abs':
                        feature_vector = np.abs(question_embedding - 
                                                candidate_sentences_embeddings[i])
                    elif q_a_combination_method == 'diff_sqr':
                        feature_vector = np.square(question_embedding - 
                                                   candidate_sentences_embeddings[i])
                    elif q_a_combination_method == 'sum':
                        feature_vector = question_embedding + candidate_sentences_embeddings[i]
                    elif q_a_combination_method == 'dot_product':
                        feature_vector = question_embedding * candidate_sentences_embeddings[i]
                    elif q_a_combination_method == 'min':
                        feature_vector = np.minimum(question_embedding,
                                                    candidate_sentences_embeddings[i])            
                    elif q_a_combination_method == 'max':
                        feature_vector = np.maximum(question_embedding,
                                                    candidate_sentences_embeddings[i])  
                        
                    train_x.append(feature_vector)
                    label = i in labels
                    train_y.append(label)
                    
    return (train_x, train_y, train_steps, train_labels)

In [5]:
get_pretrained_word_embeddings(['hello', 'world']);

load word2vec_model


In [6]:
def logistic_regression(train_x, train_y, test_x):
    pred = np.zeros(len(test_x))

    lr_model = LogisticRegression(C=1, dual=True)
    lr_model.fit(train_x, train_y)
    pred = lr_model.predict_proba(test_x)[:, 1]

    return (lr_model, pred)

In [7]:
def split_scores_for_each_question(y_pred, steps_lengths):
    pred_scores = list()
    steps_lengths_sum = 0
    
    for step_length in steps_lengths:
        scores = list()
        for idx in range(step_length):
            scores.append(y_pred[steps_lengths_sum + idx])
        
        pred_scores.append(scores)
        
        steps_lengths_sum += step_length
        
    return pred_scores

In [8]:
def run_LR_and_save_pred_scores(q_a_combination_method, data):
    data['LR'] = {'dev_preds': {}, 'train_preds': {}}
    iteration = 0
    
    for q_a_comb in q_a_combination_method:
        iteration += 1
        # feature extraction
        (train_x, train_y, train_steps, train_labels) = get_features(train, q_a_comb)
        (dev_x, dev_y, dev_steps, dev_labels) = get_features(dev, q_a_comb)
        print(q_a_comb, len(train_x[0]))
        
        # dev set
        (lr_model, dev_preds) = logistic_regression(train_x, train_y, dev_x)
        #dev_pred_scores = split_scores_for_each_question(dev_preds, dev_steps)
        
        # train set
        train_preds = lr_model.predict_proba(train_x)[:, 1]
        
        if iteration == 1:
            data['train_steps'] = train_steps
            data['dev_steps'] = dev_steps
            data['train_labels'] = train_labels
            data['dev_labels'] = dev_labels
            data['train_y'] = train_y
            data['dev_y'] = dev_y
            
        data['LR']['dev_preds'][q_a_comb] = dev_preds
        data['LR']['train_preds'][q_a_comb] = train_preds
    
    print('done')

In [9]:
data = { 
        'LR': {}, 'BM25': {}, 'string_kernels': {}, 
        'dev_steps': [], 'train_steps': [],
        'dev_labels': [], 'train_labels': [],
        'dev_y': [], 'train_y': []
       }

In [10]:
q_a_combination_method = ['diff_abs', 'diff_sqr', 
                          'dot_product', 'min', 'max']

run_LR_and_save_pred_scores(q_a_combination_method, data)

100%|██████████| 48/48 [00:03<00:00, 14.25it/s]


('diff_abs', 300)


100%|██████████| 48/48 [00:03<00:00, 14.31it/s]


('diff_sqr', 300)


100%|██████████| 48/48 [00:03<00:00, 14.77it/s]


('dot_product', 300)


100%|██████████| 48/48 [00:03<00:00, 14.29it/s]


('min', 300)


100%|██████████| 48/48 [00:03<00:00, 14.49it/s]


('max', 300)
done


In [11]:
data['LR']

{'dev_preds': {'diff_abs': array([ 0.351049  ,  0.1842451 ,  0.14537709, ...,  0.43941585,
          0.37298002,  0.40454879]),
  'diff_sqr': array([ 0.36719176,  0.2494025 ,  0.17625825, ...,  0.43176489,
          0.3483897 ,  0.42323504]),
  'dot_product': array([ 0.55856738,  0.46925643,  0.17978434, ...,  0.18840697,
          0.14716047,  0.28634511]),
  'max': array([ 0.16452467,  0.2291084 ,  0.09128775, ...,  0.6150084 ,
          0.51802607,  0.48635213]),
  'min': array([ 0.36372168,  0.19200572,  0.19769204, ...,  0.55305023,
          0.27708935,  0.43530091])},
 'train_preds': {'diff_abs': array([ 0.01598797,  0.10626231,  0.20890842, ...,  0.03472061,
          0.16273243,  0.03204314]),
  'diff_sqr': array([ 0.01084708,  0.1026302 ,  0.22100947, ...,  0.04739439,
          0.1444113 ,  0.0361476 ]),
  'dot_product': array([ 0.09637612,  0.14472967,  0.12065517, ...,  0.07932496,
          0.09597825,  0.05728984]),
  'max': array([ 0.02040813,  0.10034577,  0.11649612, 

## 3.BM25

In [12]:
from gensim.summarization.bm25 import BM25

In [13]:
def get_bm25_individual_scores(dataset):
    bm25_scores = list()
    
    for article in tqdm(dataset['data']):
        for qas_context in article['paragraphs']:
            for qas in qas_context['qas']:
                question_sentence = qas['question_words']
                candidate_sentences = qas_context['context_sentences_words']   
                
                bm25 = BM25(candidate_sentences)
                candidate_scores = bm25.get_scores(question_sentence, 1)
                bm25_scores += candidate_scores
    
    return bm25_scores

In [14]:
data['BM25'] = {'dev_preds': [], 'train_preds': []}
data['BM25']['dev_preds'] = get_bm25_individual_scores(dev)
data['BM25']['train_preds'] = get_bm25_individual_scores(train)

100%|██████████| 442/442 [00:10<00:00, 42.58it/s]


In [15]:
data['BM25']

{'dev_preds': [2.014648141364112,
  1.8460571820121379,
  1.7392427865107627,
  1.7848060348512123,
  2.014648141364112,
  1.8460571820121379,
  1.7392427865107627,
  1.7848060348512123,
  0.5801735952489722,
  0.5443634804972138,
  0,
  0.6951286261631089,
  1.5633332870641832,
  0.5443634804972138,
  0,
  1.319117192525387,
  1.772890101473316,
  1.30279883433418,
  1.082598498092505,
  3.058594780158626,
  1.3215752471733873,
  0.9235811574156969,
  0.6863426478584955,
  1.344928805108244,
  1.0314884495489012,
  0.3792176769184831,
  2.9427488291134036,
  0.9342346694266581,
  1.43447454611514,
  1.3016937015149241,
  0.3962558502340094,
  0.4656888423258251,
  1.3215752471733873,
  0.9235811574156969,
  0.6863426478584955,
  1.344928805108244,
  0.9831596918152111,
  0.9224760245964411,
  0,
  0,
  1.3215752471733873,
  0.9235811574156969,
  2.6526620314889175,
  1.344928805108244,
  0.5801735952489722,
  0.5443634804972138,
  0,
  0.6951286261631089,
  0.5801735952489722,
  0.544

## 4. String kernels

In [16]:
from string_kernels import *

In [17]:
def kernel_scores_for_a_given_question(question_words, context_sentences, kernel_type):
    scores = []
    
    for candidate_sentence in context_sentences:
        kernel_value = 0
        if kernel_type == 'spectrum_kernel':
            kernel_value = spectrum_kernel_value(question_words, candidate_sentence)
        elif kernel_type == 'presence_kernel':
            kernel_value = presence_kernel_value(question_words, candidate_sentence)
        elif kernel_type == 'intersection_kernel':
            kernel_value = intersection_kernel_value(question_words, candidate_sentence)
        
        scores.append(kernel_value)
        
    return scores

In [18]:
def get_kernel_scores(dataset, method):
    kernel_scores = list()
    
    for article in dataset['data']:
        for qas_context in article['paragraphs']:
            for qas in qas_context['qas']:
                # trying to keep the same notation
                question_words = qas['question_words']
                candidate_answers_words = qas_context['context_sentences_words']
                
                # run a method
                kernel_scores += kernel_scores_for_a_given_question(question_words, 
                                                                    candidate_answers_words,
                                                                    method)
    
    return kernel_scores

In [19]:
data['string_kernels'] = {'dev_preds': {}, 'train_preds': {}}
data['string_kernels']['dev_preds']['presence_kernel'] = get_kernel_scores(dev, 'presence_kernel')
data['string_kernels']['train_preds']['presence_kernel'] = get_kernel_scores(train, 'presence_kernel')
data['string_kernels']['dev_preds']['intersection_kernel'] = get_kernel_scores(dev, 'intersection_kernel')
data['string_kernels']['train_preds']['intersection_kernel'] = get_kernel_scores(train, 'intersection_kernel')

In [20]:
data['string_kernels']

{'dev_preds': {'intersection_kernel': [5,
   4,
   2,
   5,
   5,
   4,
   2,
   5,
   3,
   2,
   0,
   3,
   4,
   2,
   0,
   4,
   7,
   5,
   3,
   8,
   6,
   3,
   2,
   6,
   3,
   1,
   5,
   3,
   2,
   2,
   1,
   1,
   6,
   3,
   2,
   6,
   1,
   1,
   0,
   0,
   4,
   3,
   4,
   4,
   3,
   2,
   0,
   3,
   3,
   2,
   1,
   3,
   3,
   2,
   1,
   3,
   3,
   2,
   0,
   8,
   8,
   4,
   1,
   4,
   5,
   8,
   2,
   4,
   3,
   2,
   1,
   3,
   3,
   2,
   2,
   3,
   6,
   3,
   2,
   6,
   3,
   2,
   0,
   3,
   4,
   2,
   1,
   4,
   3,
   3,
   2,
   2,
   3,
   3,
   2,
   2,
   3,
   2,
   0,
   3,
   8,
   4,
   1,
   4,
   3,
   2,
   0,
   4,
   4,
   2,
   1,
   4,
   5,
   2,
   3,
   4,
   3,
   2,
   1,
   1,
   6,
   1,
   0,
   0,
   2,
   5,
   6,
   8,
   3,
   4,
   1,
   1,
   2,
   1,
   1,
   1,
   2,
   5,
   7,
   4,
   3,
   6,
   5,
   3,
   2,
   5,
   7,
   3,
   1,
   3,
   3,
   3,
   1,
   4,
   4,
   7,
   4,
   1,
   1,
   0,
   6

## 5. Classification {combined methods}

In [21]:
train_x_combined = np.vstack((data['LR']['train_preds']['diff_abs'],
                              data['LR']['train_preds']['diff_sqr'],
                              data['LR']['train_preds']['dot_product'],
                              data['LR']['train_preds']['min'],
                              data['LR']['train_preds']['max'],
                              data['BM25']['train_preds'],
                              data['string_kernels']['train_preds']['presence_kernel'],
                              data['string_kernels']['train_preds']['intersection_kernel'])).T
train_y_combined = data['train_y']
train_steps_combined = data['train_steps']
train_labels_combined = data['train_labels']

In [22]:
dev_x_combined = np.vstack((data['LR']['dev_preds']['diff_abs'],
                            data['LR']['dev_preds']['diff_sqr'],
                            data['LR']['dev_preds']['dot_product'],
                            data['LR']['dev_preds']['min'],
                            data['LR']['dev_preds']['max'],
                            data['BM25']['dev_preds'],
                            data['string_kernels']['dev_preds']['presence_kernel'],
                            data['string_kernels']['dev_preds']['intersection_kernel'])).T
dev_y_combined = data['dev_y']
dev_steps_combined = data['dev_steps']
dev_labels_combined = data['dev_labels']

## 6. Evaluation

In [23]:
def extract_scores(y_pred, steps_lengths):
    pred_scores_id = list()
    steps_lengths_sum = 0
    
    for step_length in steps_lengths:
        scores = {}
        for idx in range(step_length):
            scores[idx] = y_pred[steps_lengths_sum + idx]
        
        sorted_scores_id = sorted(scores, key=scores.get, reverse=True)
        pred_scores_id.append(sorted_scores_id)
        
        steps_lengths_sum += step_length
        
    return pred_scores_id

In [24]:
# dev set
(lr_model, dev_preds) = logistic_regression(train_x_combined,
                                            train_y_combined,
                                            dev_x_combined)
dev_pred_scores_id = extract_scores(dev_preds, dev_steps_combined)

dev_results = evaluation.get_results(dev_pred_scores_id, 
                                     dev_labels_combined, 
                                     'dev combined')
write_results(dev_results)

# train set, just for curiosity
train_preds = lr_model.predict_proba(train_x_combined)[:, 1]
train_pred_scores_id = extract_scores(train_preds, 
                                      train_steps_combined)

train_results = evaluation.get_results(train_pred_scores_id, 
                                       train_labels_combined, 
                                       'train combined')
write_results(train_results)

Method: dev combined
AvgPrec@1: 0.823746452223 (std = 0.381035739364)
MAP: 0.890315030287 (std = 0.226486792317)


Method: train combined
AvgPrec@1: 0.796036484435 (std = 0.40294217933)
MAP: 0.878285210647 (std = 0.246505459583)


